In [18]:
link1 = 'http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/cap82.txt'
link2 = 'http://people.brunel.ac.uk/~mastjjb/jeb/orlib/files/cap113.txt'
import numpy as np
import gurobipy as gp
from gurobipy import GRB

In [19]:
# define function to download data
def load_data(url, filename):
    import urllib.request
    response = urllib.request.urlretrieve(url,filename)
    file = open(filename,'r')
    return file

In [20]:
#solution encode: facility opening decision: 0 = close, 1 = open for each location
#for example: starting solution could be open all facilities
#decision = np.ones(locations)

#for a given location-opening decision, there is an optimized location-customer allocation
#allocation = np.zeros(varcost.shape)

In [21]:
#define a function to allocate transportation with a given decision
def allocate(capacity, demand, fixcost, varcost, decision):
    import numpy as np
    import gurobipy as gp
    from gurobipy import GRB
    
    model = gp.Model("Fixed-Charge Transportation problem")

    (locations,customers) = varcost.shape
    
    # Create variables
    transport = {} # transport quantity

    for n in range(locations):
        for m in range(customers):
            transport[n,m] = model.addVar(obj=varcost[n][m])

    # supply constraints
    for n in range(locations):
        model.addConstr(sum(transport[n,m] for m in range(customers)) <= capacity[n])

    # Demand constraints
    for m in range(customers):
        model.addConstr(sum(transport[n,m] for n in range(locations)) == demand[m])

    # Binding constraints
    for n in range(locations):
        for m in range(customers):
            model.addConstr(transport[n,m] <= decision[n] * min(capacity[n], demand[m]))

    model.optimize()

    solution = np.zeros(varcost.shape)
    for i in range(n):
        for j in range(m):
            solution[n,m] = transport[n,m].x

    cost = model.objVal

    model.dispose()
    return (cost, solution)



In [22]:
def feasible(capacity, demand, decision):
    if np.dot(capacity, decision) >= sum(demand):
        return True
    else:
        return False

In [23]:
def repair(capacity, demand, decision):
    sum_cap = np.dot(capacity, decision)
    sum_demand = sum(demand)
    decision_idx = [list(x) for x in enumerate(decision)]
    
    while sum_cap < sum_demand: #loop until get enough capacity to fulfill demand
        random_idx = np.random.choice([d[0] for d in decision_idx if d[1] == 0])
        decision_idx[random_idx][1] = 1
        sum_cap += capacity[decision_idx[random_idx][0]]
        
    new_decision = [d[1] for d in decision_idx]
    
    return new_decision

In [24]:
#format of a solution
def create_solution(capacity, demand, fixcost, varcost, decision):
    (vc, allocation) = allocate(capacity, demand, fixcost, varcost, decision)
    solution = [vc + np.dot(fixcost, decision), decision, allocation]
    
    return solution

In [25]:
# random initialize
# repair approach
def initialize(capacity, demand, fixcost, varcost, pop_size, infeasible = 'repair'):
    solution_pop = []
    decision_pop = []
    
    penalty = False
    penalty_cost = 100*(np.max(fixcost)*len(capacity) + np.max(varcost)*len(capacity)*len(demand)) #big cost
    
    while len(decision_pop) < pop_size: #iterate to get pop_size candidates 
        decision = list(np.random.choice([0,1], size = len(capacity)))
        
        if not feasible(capacity, demand, decision):
            if infeasible == 'reject':
                continue #continue loop without add decision
            if infeasible == 'penalty':
                penalty = True
            else: #repair
                decision = repair(capacity, demand, decision)

        if decision not in decision_pop:
            decision_pop.append(decision)
            if penalty:
                solution_pop.append([penalty_cost, decision, None])
            else: solution_pop.append(create_solution(capacity, demand, fixcost, varcost, decision))
    
    return solution_pop #return a population of decision candidates

In [26]:
def selection_roulette(solution_pop, offspring_size):
    
    expected = np.mean([ sol[0] for sol in solution_pop])
    
    pop_size = len(solution_pop)
    f = [ (2*expected - solution_pop[i][0]) for i in range(pop_size)] #calculate f of each candidate
    sum_f = sum(f)
    prob_range = [ sum(f[0:i+1])/sum_f for i in range(pop_size)] #cumulative prob of each candidate
    
    offspring = []
    for i in range(offspring_size):
        p = np.random.random()
        for j in range(pop_size):
            if prob_range[j] >= p:
                offspring.append(solution_pop[j])
                break
    
    return offspring

In [27]:
#recombination and mutation
def recomb_uniform(capacity, demand, fixcost, varcost, solution_pop, offspring,
                   recprob, muprob, crossprob = 0.5, mulocprob = 0.5, infeasible = "repair"):
    locations = len(offspring[0][1])
    penalty_cost = 100*(np.max(fixcost)*len(capacity) + np.max(varcost)*len(capacity)*len(demand)) #big cost
    
    pop = [s[1] for s in solution_pop] #set of decision
    
    for k in range(len(offspring)//2): 
        penalty = False

        #recombination
        if np.random.random() < recprob:
            child = []
            #uniform crossover
            for i in range(locations):
                if np.random.random() < crossprob:
                    child.append(offspring[2*k][1][i]) #from offspring1
                else: 
                    child.append(offspring[2*k+1][1][i]) #from offspring2
    
            #mutation
            if np.random.random() < muprob:
                for i in range(locations):
                    if np.random.random() < mulocprob:
                        child[i] = abs(child[i] - 1)

            #infeasible solution
            if not feasible(capacity, demand, child):
                if infeasible == 'reject':
                    continue #continue loop without add decision
                if infeasible == 'penalty':
                    penalty = True
                else: #repair
                    decision = repair(capacity, demand, child)
        

            if child not in pop:
                if penalty:
                    solution_pop.append([penalty_cost, child, None])
                else: solution_pop.append(create_solution(capacity, demand, fixcost, varcost, child))
            
    return solution_pop

In [28]:
# replacement
def replace_best(solution_pop, pop_size):
    sortedpop = sorted(solution_pop)
    pop = sortedpop[0:pop_size]
    
    return pop 

In [29]:
file = load_data(link2, 'cap113.txt')
data = file.readlines()

In [30]:
[locations, customers] = [int(x) for x in data[0].split(' ')[1:-1]]
capacity = []
fixcost = []
for i in range(locations):
    [c, f] = [float(x) for x in data[i+1].split(' ')[1:-1]]
    capacity.append(c)
    fixcost.append(f)
demand = []
varcost = np.zeros((locations,customers))
for j in range(customers):
    demand.append(float(data[j*9 + (1+locations)].split(' ')[1]))
    v = list(np.concatenate([data[j*9 + (1+locations+t)].split(' ')[1:-1] for t in range(1,9)]))
    for i in range(locations):
        varcost[i,j] = float(v[i])/demand[j] #cost of allocating demand of j to warehouse i per unit
        
        
print(locations)
print(customers)
print(capacity)
print(fixcost)
print(demand)
print(varcost)


50
50
[5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0, 5000.0]
[17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 0.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0, 17500.0]
[146.0, 87.0, 672.0, 1337.0, 31.0, 559.0, 2370.0, 1089.0, 33.0, 32.0, 5495.0, 904.0, 1466.0, 143.0, 615.0, 564.0, 226.0, 3016.0, 253.0, 195.0, 38.

In [31]:
#parameters
pop_size = 10
offspring_size = 50
iterations = 5

In [34]:
#main algorithm    
solution_pop = initialize(capacity, demand, fixcost, varcost, pop_size)

for it in range(iterations):
    offspring = selection_roulette(solution_pop, offspring_size)
    solution_pop = recomb_uniform(capacity, demand, fixcost, varcost, solution_pop, offspring,
                       recprob = 0.7, muprob = 0.3, crossprob = 0.5, mulocprob = 0.5, infeasible = 'repair')
    solution_pop = replace_best(solution_pop, pop_size)
    
print(solution_pop[0])

Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2600 rows, 2500 columns and 7500 nonzeros
Model fingerprint: 0xd4438749
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]
Presolve removed 2527 rows and 1350 columns
Presolve time: 0.00s
Presolved: 73 rows, 1150 columns, 2300 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.3458645e+05   3.238000e+03   0.000000e+00      0s
      15    8.1603750e+05   0.000000e+00   0.000000e+00      0s

Solved in 15 iterations and 0.01 seconds
Optimal objective  8.160375000e+05
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2600 rows, 2500 columns and 7500 nonzeros
Model fingerprint: 0x2dc86130
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]


Solved in 19 iterations and 0.01 seconds
Optimal objective  8.617118125e+05
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2600 rows, 2500 columns and 7500 nonzeros
Model fingerprint: 0x5c08e809
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]
Presolve removed 2525 rows and 1250 columns
Presolve time: 0.01s
Presolved: 75 rows, 1250 columns, 2500 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0454416e+05   2.550000e+03   0.000000e+00      0s
      10    7.5675955e+05   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds
Optimal objective  7.567595500e+05
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2600 rows, 2500 columns and 7500 nonzeros
Model fingerprint: 0x689b2e4b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+0

      12    7.4159273e+05   0.000000e+00   0.000000e+00      0s

Solved in 12 iterations and 0.01 seconds
Optimal objective  7.415927250e+05
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2600 rows, 2500 columns and 7500 nonzeros
Model fingerprint: 0xb239f55c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]
Presolve removed 2526 rows and 1300 columns
Presolve time: 0.01s
Presolved: 74 rows, 1200 columns, 2400 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5971212e+05   2.640875e+03   0.000000e+00      0s
      16    8.4369075e+05   0.000000e+00   0.000000e+00      0s

Solved in 16 iterations and 0.01 seconds
Optimal objective  8.436907500e+05
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2600 rows, 2500 columns and 7500 nonzeros
Model fingerprint: 0x45366163
Coefficient statis

       0    7.1458221e+05   3.351000e+03   0.000000e+00      0s
      20    8.3303205e+05   0.000000e+00   0.000000e+00      0s

Solved in 20 iterations and 0.01 seconds
Optimal objective  8.330320500e+05
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2600 rows, 2500 columns and 7500 nonzeros
Model fingerprint: 0x8afa39d8
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]
Presolve removed 2524 rows and 1200 columns
Presolve time: 0.01s
Presolved: 76 rows, 1300 columns, 2600 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0977040e+05   2.183000e+03   0.000000e+00      0s
      10    7.5279010e+05   0.000000e+00   0.000000e+00      0s

Solved in 10 iterations and 0.01 seconds
Optimal objective  7.527901000e+05
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2600 rows, 2500 columns an


Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.9240404e+05   2.314000e+03   0.000000e+00      0s
      11    7.6083916e+05   0.000000e+00   0.000000e+00      0s

Solved in 11 iterations and 0.01 seconds
Optimal objective  7.608391625e+05
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2600 rows, 2500 columns and 7500 nonzeros
Model fingerprint: 0x0c53ff79
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]
Presolve removed 2524 rows and 1200 columns
Presolve time: 0.01s
Presolved: 76 rows, 1300 columns, 2600 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0121830e+05   2.131000e+03   0.000000e+00      0s
       9    7.4415660e+05   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds
Optimal objective  7.441566000e+05
Gurobi Optimizer version 9.0.2 build v9

Presolve time: 0.01s
Presolved: 75 rows, 1250 columns, 2500 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.5330694e+05   2.731000e+03   0.000000e+00      0s
       9    7.8327738e+05   0.000000e+00   0.000000e+00      0s

Solved in 9 iterations and 0.01 seconds
Optimal objective  7.832773750e+05
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2600 rows, 2500 columns and 7500 nonzeros
Model fingerprint: 0xf5ff339c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]
Presolve removed 2527 rows and 1350 columns
Presolve time: 0.01s
Presolved: 73 rows, 1150 columns, 2300 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.2493947e+05   2.721000e+03   0.000000e+00      0s
      21    8.0282144e+05   0.000000e+00   0.000000e+00      0s

Solved in 21 iterations and 0.01 seconds
Optim

  RHS range        [3e+01, 1e+04]
Presolve removed 2527 rows and 1350 columns
Presolve time: 0.01s
Presolved: 73 rows, 1150 columns, 2300 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.0398939e+05   2.540875e+03   0.000000e+00      0s
      16    7.7918216e+05   0.000000e+00   0.000000e+00      0s

Solved in 16 iterations and 0.01 seconds
Optimal objective  7.791821625e+05
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2600 rows, 2500 columns and 7500 nonzeros
Model fingerprint: 0xde49c04b
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]
Presolve removed 2524 rows and 1200 columns
Presolve time: 0.01s
Presolved: 76 rows, 1300 columns, 2600 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.1114261e+05   2.131000e+03   0.000000e+00      0s
       9    7.4958721e+05   0.00

  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]
Presolve removed 2526 rows and 1300 columns
Presolve time: 0.01s
Presolved: 74 rows, 1200 columns, 2400 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.3020810e+05   3.455125e+03   0.000000e+00      0s
      23    8.8898244e+05   0.000000e+00   0.000000e+00      0s

Solved in 23 iterations and 0.01 seconds
Optimal objective  8.889824375e+05
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2600 rows, 2500 columns and 7500 nonzeros
Model fingerprint: 0x06f37e49
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]
Presolve removed 2521 rows and 1050 columns
Presolve time: 0.01s
Presolved: 79 rows, 1450 columns, 2900 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.     

Model fingerprint: 0x47d96438
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]
Presolve removed 2526 rows and 1300 columns
Presolve time: 0.01s
Presolved: 74 rows, 1200 columns, 2400 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    7.9531238e+05   3.576000e+03   0.000000e+00      0s
      30    8.5845383e+05   0.000000e+00   0.000000e+00      0s

Solved in 30 iterations and 0.01 seconds
Optimal objective  8.584538250e+05
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 2600 rows, 2500 columns and 7500 nonzeros
Model fingerprint: 0xe36f4f48
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 1e+04]
Presolve removed 2528 rows and 1400 columns
Presolve time: 0.00s
Presolved: 72 rows, 1100 columns, 2200 nonzeros

Iter